<a href="https://colab.research.google.com/github/Shibu4064/ML_Project/blob/main/test_evaluation_sumon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers
!pip install -q datasets
!pip install transformers[torch]
!pip install transformers[tf-cpu]
!pip install transformers[flax]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 73.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.6/225.6 MB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.7/454.7 kB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import multiprocessing
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
import transformers
import torch.nn as nn
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import AutoModelForMaskedLM, AutoModelForPreTraining, AutoModelForSequenceClassification, EarlyStoppingCallback
from transformers import AutoTokenizer, AutoConfig
from transformers import BertForMaskedLM, DistilBertForMaskedLM
from transformers import BertTokenizer, DistilBertTokenizer
from transformers import RobertaTokenizer, RobertaForMaskedLM
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorForLanguageModeling
from tokenizers import BertWordPieceTokenizer
from transformers import TextClassificationPipeline
import gdown

In [3]:
import gdown

dataset_link = "https://drive.google.com/uc?id=1o4WpF71L5PNLBEl0HeTp13V35GZjEd5q"

# ami apatoto ekhane train dataset er drive link er id disi
# test dataset amar kase nai test dataset drive e upload diye share kore then oi id ta eikhane boshai dite hobe

gdown.download(dataset_link,"test.csv")



Downloading...
From: https://drive.google.com/uc?id=1o4WpF71L5PNLBEl0HeTp13V35GZjEd5q
To: /content/test.csv
100%|██████████| 607k/607k [00:00<00:00, 24.2MB/s]


'test.csv'

In [4]:
import gdown

checkpoint_link = "https://drive.google.com/uc?id=1-hNxX7itJapRTH9OzZc4l2IToK9RH11v"

gdown.download(checkpoint_link,"checkpoint.zip")

Downloading...
From: https://drive.google.com/uc?id=1-hNxX7itJapRTH9OzZc4l2IToK9RH11v
To: /content/checkpoint.zip
100%|██████████| 761M/761M [00:16<00:00, 47.3MB/s]


'checkpoint.zip'

In [5]:
!unzip checkpoint.zip

Archive:  checkpoint.zip
   creating: content/training_with_callbacks/checkpoint-9500/
  inflating: content/training_with_callbacks/checkpoint-9500/vocab.txt  
  inflating: content/training_with_callbacks/checkpoint-9500/optimizer.pt  
  inflating: content/training_with_callbacks/checkpoint-9500/special_tokens_map.json  
  inflating: content/training_with_callbacks/checkpoint-9500/scheduler.pt  
  inflating: content/training_with_callbacks/checkpoint-9500/rng_state.pth  
  inflating: content/training_with_callbacks/checkpoint-9500/trainer_state.json  
  inflating: content/training_with_callbacks/checkpoint-9500/tokenizer_config.json  
  inflating: content/training_with_callbacks/checkpoint-9500/tokenizer.json  
  inflating: content/training_with_callbacks/checkpoint-9500/config.json  
  inflating: content/training_with_callbacks/checkpoint-9500/training_args.bin  
  inflating: content/training_with_callbacks/checkpoint-9500/pytorch_model.bin  


In [6]:
from transformers import AutoTokenizer, AutoModel, AutoConfig

model_name = "csebuetnlp/banglabert"

tokenizer = AutoTokenizer.from_pretrained(model_name)

config = AutoConfig.from_pretrained("/content/content/training_with_callbacks/checkpoint-9500/config.json")

model = AutoModelForSequenceClassification.from_pretrained("/content/content/training_with_callbacks/checkpoint-9500/pytorch_model.bin",config=config)

model

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): L

In [7]:
df_test = pd.read_csv("test.csv")

In [9]:
df_test.head(5)

,text
0,বাংলাদেশের হিন্দুরা নিজেদের জন্য আলাদা হিন্দু ...
1,মাইজদী - চৌমুহুনী - ফেনী মন্দিরে হামলা নিয়ে রি...
2,"দয়া করে পবিত্র কুরআনুল কারিম বলেন,,,,পবিত্র কথ..."
3,বিবিসি হলো সত্য কে বিনষ্টকারী আর মিথ্যা কে গ্র...
4,বুধবার কি তোরা মারা গেছিলি বিবিসি বাংলা


In [10]:
pipeline = TextClassificationPipeline(model=model,tokenizer=tokenizer,return_all_scores=True)
output = pipeline(df_test["text"].tolist())
print(output)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:105: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


[[{'label': 'LABEL_0', 'score': 0.012520099990069866}, {'label': 'LABEL_1', 'score': 0.002743225544691086}, {'label': 'LABEL_2', 'score': 0.9847366213798523}], [{'label': 'LABEL_0', 'score': 0.9997602105140686}, {'label': 'LABEL_1', 'score': 0.00021427765022963285}, {'label': 'LABEL_2', 'score': 2.548558040871285e-05}], [{'label': 'LABEL_0', 'score': 0.999692440032959}, {'label': 'LABEL_1', 'score': 0.0002769018756225705}, {'label': 'LABEL_2', 'score': 3.062538962694816e-05}], [{'label': 'LABEL_0', 'score': 0.002017261926084757}, {'label': 'LABEL_1', 'score': 0.9976145029067993}, {'label': 'LABEL_2', 'score': 0.0003683642717078328}], [{'label': 'LABEL_0', 'score': 0.9975284934043884}, {'label': 'LABEL_1', 'score': 0.002282382221892476}, {'label': 'LABEL_2', 'score': 0.0001892286672955379}], [{'label': 'LABEL_0', 'score': 0.002345758955925703}, {'label': 'LABEL_1', 'score': 0.9971601963043213}, {'label': 'LABEL_2', 'score': 0.0004940173821523786}], [{'label': 'LABEL_0', 'score': 0.99928

In [11]:
results = output

In [12]:
for res in results:
  print(res)

[{'label': 'LABEL_0', 'score': 0.012520099990069866}, {'label': 'LABEL_1', 'score': 0.002743225544691086}, {'label': 'LABEL_2', 'score': 0.9847366213798523}]
[{'label': 'LABEL_0', 'score': 0.9997602105140686}, {'label': 'LABEL_1', 'score': 0.00021427765022963285}, {'label': 'LABEL_2', 'score': 2.548558040871285e-05}]
[{'label': 'LABEL_0', 'score': 0.999692440032959}, {'label': 'LABEL_1', 'score': 0.0002769018756225705}, {'label': 'LABEL_2', 'score': 3.062538962694816e-05}]
[{'label': 'LABEL_0', 'score': 0.002017261926084757}, {'label': 'LABEL_1', 'score': 0.9976145029067993}, {'label': 'LABEL_2', 'score': 0.0003683642717078328}]
[{'label': 'LABEL_0', 'score': 0.9975284934043884}, {'label': 'LABEL_1', 'score': 0.002282382221892476}, {'label': 'LABEL_2', 'score': 0.0001892286672955379}]
[{'label': 'LABEL_0', 'score': 0.002345758955925703}, {'label': 'LABEL_1', 'score': 0.9971601963043213}, {'label': 'LABEL_2', 'score': 0.0004940173821523786}]
[{'label': 'LABEL_0', 'score': 0.999280631542

In [13]:
labels = []
for res in results:

  res = [r["score"] for r in res]

  res = np.array(res)
  label = np.argmax(res)
  labels.append(label)

In [14]:
df_test['label'] = labels

In [15]:
df_test.to_csv("test_predicted.csv")

In [16]:
# df_test.groupby("label").count()
df_test  = df_test.reset_index(drop=True)
# df_test[['text','label']]
df_test.to_csv("test_predicted.csv",index=False)

In [17]:
df_test = df_test[["text","label"]]
df_test.to_csv("test_pred.csv",index=False)

In [18]:
df_test.groupby("label").count()

,text
label,
0,1170
1,486
2,360


In [19]:
df2=pd.read_csv("/content/test_pred.csv")
df2.head()

,text,label
0,বাংলাদেশের হিন্দুরা নিজেদের জন্য আলাদা হিন্দু ...,2
1,মাইজদী - চৌমুহুনী - ফেনী মন্দিরে হামলা নিয়ে রি...,0
2,"দয়া করে পবিত্র কুরআনুল কারিম বলেন,,,,পবিত্র কথ...",0
3,বিবিসি হলো সত্য কে বিনষ্টকারী আর মিথ্যা কে গ্র...,1
4,বুধবার কি তোরা মারা গেছিলি বিবিসি বাংলা,0
